In [1]:
import torch
import torch_geometric
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import OptPairTensor, Adj, Size
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import GATConv
from torch_geometric.loader import NeighborLoader

import os
import argparse
import time
from datetime import datetime
import random
from typing import Union, Tuple

import numpy as np
import pandas as pd
from scipy.sparse import load_npz
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve, auc
from scipy.stats import pearsonr

from all_models import GCN_regression, GCN_classification, GCN_classification_weighted, MLP_regression, MLP_classification, GAT

### Classification Models

#### Helper Functions to train/evaluate models

In [2]:
def to_cpu_npy(x):
    return x.cpu().detach().numpy()

def train_model_classification(model, graph, max_epoch, learning_rate, targetNode_mask, train_idx, valid_idx, optimizer):
    '''
    Trains model for classification task
    
    Parameters
    ----------
    model [GCN_classification]: Instantiation of model class
    graph [PyG Data class]: PyTorch Geometric Data object representing the graph
    max_epoch [int]: Maximum number of training epochs
    learning_rate [float]: Learning rate
    targetNode_mask [tensor]: Subgraph mask for training nodes
    train_idx [array]: Node IDs corresponding to training set
    valid_idx [array]: Node IDs corresponding to validation set
    optimizer [PyTorch optimizer class]: PyTorch optimization algorithm

    Returns
    -------
    train_loss_vec [array]: Training loss for each epoch
    train_AUROC_vec [array]: Training AUROC score for each epoch
    valid_loss_vec [array]: Validation loss for each epoch
    valid_AUROC_vec [array]: Validation AUROC score for each epoch

    '''
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    graph = graph.to(device)

    optimizer = optimizer
    
    train_labels = to_cpu_npy(graph.y[targetNode_mask[train_idx]])
    valid_labels = to_cpu_npy(graph.y[targetNode_mask[valid_idx]])
    
    train_loss_list = []
    train_AUROC_vec = np.zeros(np.shape(np.arange(max_epoch)))
    valid_loss_list = []
    valid_AUROC_vec = np.zeros(np.shape(np.arange(max_epoch)))

    model.train()
    train_status = True
    
    print('\n')
    for e in list(range(max_epoch)):
        
        if e%100 == 0:
            print("Epoch", str(e), 'out of', str(max_epoch))
        
        model.train()
        train_status = True
        
        optimizer.zero_grad()
        
        ### Only trains on nodes with genes due to masking
        forward_scores = model(graph.x.float(), graph.edge_index, train_status)[targetNode_mask]
        
        train_scores = forward_scores[train_idx]

        train_loss  = model.loss(train_scores, torch.LongTensor(train_labels).to(device))

        train_softmax, _ = model.calc_softmax_pred(train_scores)

        train_loss.backward()
        
        optimizer.step()
            
        ### Calculate training and validation loss, AUROC scores
        model.eval()
        
        valid_scores = forward_scores[valid_idx]
        valid_loss  = model.loss(valid_scores, torch.LongTensor(valid_labels).to(device))
        valid_softmax, _ = model.calc_softmax_pred(valid_scores) 

        train_loss_list.append(train_loss.item())
        train_softmax = to_cpu_npy(train_softmax)
        train_AUROC = roc_auc_score(train_labels, train_softmax[:,1], average="micro")

        valid_loss_list.append(valid_loss.item())
        valid_softmax = to_cpu_npy(valid_softmax)
        valid_AUROC = roc_auc_score(valid_labels, valid_softmax[:,1], average="micro")
        
        train_AUROC_vec[e] = train_AUROC
        valid_AUROC_vec[e] = valid_AUROC

    train_loss_vec = np.reshape(np.array(train_loss_list), (-1, 1))
    valid_loss_vec = np.reshape(np.array(valid_loss_list), (-1, 1))

    return train_loss_vec, train_AUROC_vec, valid_loss_vec, valid_AUROC_vec


def eval_model_classification(model, graph, targetNode_mask, train_idx, valid_idx, test_idx):
    '''
    Runs fully trained classification model and compute evaluation statistics

    Parameters
    ----------
    model [GCN_classification]: Instantiation of model class
    graph [PyG Data class]: PyTorch Geometric Data object representing the graph
    targetNode_mask [tensor]: Mask ensuring model only trains on nodes with genes
    train_idx [array]: Node IDs corresponding to training set;
        analogous for valid_idx and test_idx

    Returns
    -------
    test_AUROC [float]: Test set AUROC score;
        analogous for train_AUROC (training set) and valid_AUPR (validation set)
    test_AUPR [float]: Test set AUPR score
        analogous for train_AUPR (training set) and valid_AUPR (validation set)
    test_pred [array]: Test set predictions;
        analogous for train_pred (training set) and valid_pred (validation set)
    test_labels [array]: Test set labels;
        analagous for train_labels (training set) and valid_labels (validation set)
    '''
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    graph = graph.to(device)
    test_labels = to_cpu_npy(graph.y[targetNode_mask[test_idx]])
    
    model.eval()
    train_status=False

    forward_scores = model(graph.x.float(), graph.edge_index, train_status)[targetNode_mask]

    test_scores = forward_scores[test_idx]
    test_softmax, test_pred = model.calc_softmax_pred(test_scores) 
    
    test_softmax = to_cpu_npy(test_softmax)
    test_pred = to_cpu_npy(test_pred)
    test_AUROC = roc_auc_score(test_labels, test_softmax[:,1], average="micro")
    test_precision, test_recall, thresholds = precision_recall_curve(test_labels, test_softmax[:,1])
    test_AUPR = auc(test_recall, test_precision)
    # test_F1 = f1_score(test_labels, test_pred, average="micro")
    
    train_scores = forward_scores[train_idx]
    train_labels = to_cpu_npy(graph.y[targetNode_mask[train_idx]])
    train_softmax, train_pred = model.calc_softmax_pred(train_scores) 
    train_pred = to_cpu_npy(train_pred)
    train_softmax = to_cpu_npy(train_softmax)
    train_precision, train_recall, thresholds = precision_recall_curve(train_labels, train_softmax[:,1])
    train_AUPR = auc(train_recall, train_precision)
    # train_F1 = f1_score(train_labels, train_pred, average="micro")

    valid_scores = forward_scores[valid_idx]
    valid_labels = to_cpu_npy(graph.y[targetNode_mask[valid_idx]])
    valid_softmax, valid_pred = model.calc_softmax_pred(valid_scores) 
    valid_pred = to_cpu_npy(valid_pred)
    valid_softmax = to_cpu_npy(valid_softmax)
    valid_precision, valid_recall, thresholds = precision_recall_curve(valid_labels, valid_softmax[:,1])
    valid_AUPR = auc(valid_recall, valid_precision)
    # valid_F1 = f1_score(valid_labels, valid_pred, average="micro")

    return test_AUROC, test_AUPR, test_pred, test_labels, train_AUPR, train_pred, train_labels, \
        valid_AUPR, valid_pred, valid_labels


In [3]:
def train_model_classification_GAT(model, loss, graph, max_epoch, learning_rate, targetNode_mask, train_idx, valid_idx, optimizer):
    model = model.to(device)
    graph = graph.to(device)

    optimizer = optimizer
    
    train_labels = to_cpu_npy(graph.y[targetNode_mask[train_idx]])
    valid_labels = to_cpu_npy(graph.y[targetNode_mask[valid_idx]])
    
    model.train()
    train_status = True
    
    print('\n')

    train_losses = []
    valid_losses = []
    for e in list(range(max_epoch)):
        model.train()
        optimizer.zero_grad()

        all_scores = model(graph)[targetNode_mask]
        train_scores = all_scores[train_idx]
        
        train_loss = loss(train_scores, torch.LongTensor(train_labels).to(device))
        train_losses.append(train_loss.item())

        train_loss.backward()
        optimizer.step()

        model.eval()
        valid_scores = all_scores[valid_idx]
        valid_loss = loss(valid_scores, torch.LongTensor(valid_labels).to(device))
        valid_losses.append(valid_loss.item())

        if e%100 == 0:
            print(f'Epoch {e}: Train Loss = {train_loss}, Valid Loss = {valid_loss}')

    return train_losses, valid_losses

def eval_model_classification_GAT(model, graph, targetNode_mask, train_idx, valid_idx, test_idx):
    model = model.to(device)
    graph = graph.to(device)
    test_labels = to_cpu_npy(graph.y[targetNode_mask[test_idx]])
    
    model.eval()

    forward_scores = model(G)[targetNode_mask]

    test_scores = forward_scores[test_idx]
    test_softmax = F.softmax(test_scores, dim=1)
    test_preds = torch.argmax(test_softmax, dim=1)
    
    test_softmax = to_cpu_npy(test_softmax)
    test_preds = to_cpu_npy(test_preds)
    test_AUROC = roc_auc_score(test_labels, test_softmax[:,1], average="micro")
    test_acc = np.mean(test_preds == test_labels)

    train_labels = to_cpu_npy(graph.y[targetNode_mask[train_idx]])
    train_scores = forward_scores[train_idx]
    train_softmax = F.softmax(train_scores, dim=1)
    train_preds = torch.argmax(train_softmax, dim=1)
    
    train_softmax = to_cpu_npy(train_softmax)
    train_preds = to_cpu_npy(train_preds)
    train_AUROC = roc_auc_score(train_labels, train_softmax[:,1], average="micro")
    train_acc = np.mean(train_preds == train_labels)


    return {'train_AUROC': train_AUROC, 'train_acc': train_acc, 'test_AUROC': test_AUROC, 'test_acc': test_acc}

In [4]:
def train_model_classification_GAT_Neighbors(model, loss, train_loader, valid_loader, max_epoch, optimizer, train_idx, valid_idx):
    model = model.to(device)

    train_losses = []
    valid_losses = []
    for epoch in range(10):
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            train_batch_mask = torch.isin(batch.n_id.to(device), targetNode_mask.to(device))
            train_batch_scores = model(batch)[train_batch_mask]
            train_batch_labels = to_cpu_npy(batch.y[train_batch_mask])
            train_batch_loss = loss(train_batch_scores, torch.LongTensor(train_batch_labels).to(device))
            train_losses.append(train_batch_loss.item())
            train_batch_loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.no_grad():
            for batch in valid_loader:
                batch = batch.to(device)
                valid_batch_mask = torch.isin(batch.n_id.to(device), targetNode_mask.to(device))
                valid_batch_scores = model(batch)[valid_batch_mask]
                valid_batch_labels = to_cpu_npy(batch.y[valid_batch_mask])
                valid_batch_loss = loss(valid_batch_scores, torch.LongTensor(valid_batch_labels).to(device))
                valid_losses.append(valid_batch_loss.item())
                
        print(f'Epoch {epoch}: Train Loss = {train_batch_loss}, Valid Loss = {valid_batch_loss}')
    return train_losses, valid_losses

#### GC-Merge Classification

In [5]:
cell_line = 'E116'
max_epoch = 1000
learning_rate = 1e-4
num_graph_conv_layers = 2
graph_conv_embed_size = 256
num_lin_layers = 3
lin_hidden_size = 256
regression_flag = 0
random_seed = 0

chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)
num_classes = 2 if regression_flag == 0 else 1

In [6]:
src_dir = os.getcwd()
save_dir = os.path.join(src_dir, 'data', cell_line, 'saved_runs')
hic_sparse_mat_file = os.path.join(src_dir, 'data', cell_line, 'hic_sparse.npz')
np_nodes_lab_genes_file = os.path.join(src_dir, 'data',  cell_line, \
    'np_nodes_lab_genes_reg' + str(regression_flag) + '.npy')
np_hmods_norm_all_file = os.path.join(src_dir, 'data', cell_line, \
    'np_hmods_norm_chip_' + str(chip_res) + 'bp.npy')
df_genes_file = os.path.join(src_dir, 'data', cell_line, 'df_genes_reg' + str(regression_flag) + '.pkl')
df_genes = pd.read_pickle(df_genes_file)

mat = load_npz(hic_sparse_mat_file)
allNodes_hms = np.load(np_hmods_norm_all_file)
hms = allNodes_hms[:, 1:] #only includes features, not node ids
X = torch.tensor(hms).float().reshape(-1, num_feat) 
allNodes = allNodes_hms[:, 0].astype(int)
geneNodes_labs = np.load(np_nodes_lab_genes_file)

geneNodes = geneNodes_labs[:, -2].astype(int)
allLabs = -1*np.ones(np.shape(allNodes))

targetNode_mask = torch.tensor(geneNodes).long()

if regression_flag == 0:
    geneLabs = geneNodes_labs[:, -1].astype(int)
    allLabs[geneNodes] = geneLabs
    Y = torch.tensor(allLabs).long()
else:
    geneLabs = geneNodes_labs[:, -1].astype(float)
    allLabs[geneNodes] = geneLabs
    Y = torch.tensor(allLabs).float()

In [7]:
#hyperparameter and model definitions
max_epoch = 1000
learning_rate = 1e-4
num_graph_conv_layers = 2
graph_conv_embed_size = 256
num_lin_layers = 3
lin_hidden_size = 256
regression_flag = 0
random_seed = 0

chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)
num_classes = 2 if regression_flag == 0 else 1

print('ChIP-seq resolution:', str(chip_res))
print('\n')
print('Training set: 70%')
print('Validation set: 15%')
print('Testing set: 15%')
print('\n')
print('Model hyperparameters: ')
print('Number of epochs:', max_epoch)
print('Learning rate:', learning_rate)
print('Number of graph convolutional layers:', str(num_graph_conv_layers))
print('Graph convolutional embedding size:', graph_conv_embed_size)
print('Number of linear layers:', str(num_lin_layers))
print('Linear hidden layer size:', lin_hidden_size)

# random_seed = random.randint(0,10000)
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

###Test for GPU availability
cuda_flag = torch.cuda.is_available()
if cuda_flag:  
  dev = "cuda" 
else:
  dev = "cpu"  
device = torch.device(dev)

ChIP-seq resolution: 10000


Training set: 70%
Validation set: 15%
Testing set: 15%


Model hyperparameters: 
Number of epochs: 1000
Learning rate: 0.0001
Number of graph convolutional layers: 2
Graph convolutional embedding size: 256
Number of linear layers: 3
Linear hidden layer size: 256


In [8]:
###Load input files
def prepare_data(cell_line, regression_flag, base_path):
    save_dir = os.path.join(base_path, 'data', cell_line, 'saved_runs')
    hic_sparse_mat_file = os.path.join(base_path, 'data', cell_line, 'hic_sparse.npz')
    np_nodes_lab_genes_file = os.path.join(base_path, 'data',  cell_line, \
        'np_nodes_lab_genes_reg' + str(regression_flag) + '.npy')
    np_hmods_norm_all_file = os.path.join(base_path, 'data', cell_line, \
        'np_hmods_norm_chip_' + str(chip_res) + 'bp.npy')
    df_genes_file = os.path.join(base_path, 'data', cell_line, 'df_genes_reg' + str(regression_flag) + '.pkl')
    df_genes = pd.read_pickle(df_genes_file)
    
    mat = load_npz(hic_sparse_mat_file)
    allNodes_hms = np.load(np_hmods_norm_all_file)
    hms = allNodes_hms[:, 1:] #only includes features, not node ids
    X = torch.tensor(hms).float().reshape(-1, num_feat) 
    allNodes = allNodes_hms[:, 0].astype(int)
    geneNodes_labs = np.load(np_nodes_lab_genes_file)

    geneNodes = geneNodes_labs[:, -2].astype(int)
    allLabs = -1*np.ones(np.shape(allNodes))

    targetNode_mask = torch.tensor(geneNodes).long()

    if regression_flag == 0:
        geneLabs = geneNodes_labs[:, -1].astype(int)
        allLabs[geneNodes] = geneLabs
        Y = torch.tensor(allLabs).long()
    else:
        geneLabs = geneNodes_labs[:, -1].astype(float)
        allLabs[geneNodes] = geneLabs
        Y = torch.tensor(allLabs).float()

    extract = torch_geometric.utils.from_scipy_sparse_matrix(mat)
    data = torch_geometric.data.Data(edge_index = extract[0], edge_attr = extract[1], x = X, y = Y)
    G = data
    
    ###Randomize node order and split into 70%/15%/15% training/validation/test sets
    torch.manual_seed(0)
    pred_idx_shuff = torch.randperm(targetNode_mask.shape[0])
    fin_train = np.floor(0.7*pred_idx_shuff.shape[0]).astype(int)
    fin_valid = np.floor(0.85*pred_idx_shuff.shape[0]).astype(int)
    
    return G, targetNode_mask

In [9]:
graph_conv_embed_size = 256
num_graph_conv_layers = 2

graph_conv_layer_sizes = [num_feat] + \
    [int(max(graph_conv_embed_size, lin_hidden_size)) \
          for i in np.arange(1, num_graph_conv_layers, 1)] + [lin_hidden_size]

graph_lin_hidden_sizes = [graph_conv_layer_sizes[-1]] + \
    [int(max(lin_hidden_size, num_classes)) \
          for i in np.arange(1, num_lin_layers, 1)] + [num_classes]

graph_lin_hidden_sizes_reg = [graph_conv_layer_sizes[-1]] + \
    [int(max(lin_hidden_size, num_classes)) \
          for i in np.arange(1, num_lin_layers, 1)] + [1]

In [10]:
num_classes_reg = 1
lin_hidden_sizes = [num_feat] + [int(max(lin_hidden_size, num_classes)) for i in np.arange(1, num_lin_layers, 1)] + [num_classes]
lin_hidden_sizes_reg = [num_feat] + [int(max(lin_hidden_size, num_classes_reg)) for i in np.arange(1, num_lin_layers, 1)] + [num_classes_reg]
cell_lines = ['E116', 'E122', 'E123']

print(lin_hidden_sizes)

[6, 256, 256, 2]


### GCN

In [11]:
max_epoch = 1000
gcn_auroc = []

for cell_line in cell_lines:
    print(f'\nTraining Cell Line {cell_line}...')
    
    train_idx = torch.load(f'./train-test-split/{cell_line}/train_idx.pt')
    valid_idx = torch.load(f'./train-test-split/{cell_line}/valid_idx.pt')
    test_idx = torch.load(f'./train-test-split/{cell_line}/test_idx.pt')
    
    G, targetNode_mask = prepare_data(cell_line=cell_line, regression_flag = regression_flag, base_path = src_dir)
    
    model = GCN_classification(num_feat, num_graph_conv_layers, graph_conv_layer_sizes, num_lin_layers, graph_lin_hidden_sizes, num_classes)
    optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, model.parameters()), lr = learning_rate)

    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    print(f'Number of Parameters: {sum([np.prod(p.size()) for p in model_parameters])}')

    train_loss_vec, train_AUROC_vec, valid_loss_vec, valid_AUROC_vec = \
    train_model_classification(model, G, max_epoch, learning_rate, targetNode_mask, train_idx, valid_idx, optimizer)

    test_AUROC, test_AUPR, test_pred, test_labels, train_AUPR, train_pred, train_labels, \
        valid_AUPR, valid_pred, valid_labels = \
            eval_model_classification(model, G, targetNode_mask, train_idx, valid_idx, test_idx)
    
    gcn_auroc.append(test_AUROC)


Training Cell Line E116...
Number of Parameters: 266754


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000

Training Cell Line E122...
Number of Parameters: 266754


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000

Training Cell Line E123...
Number of Parameters: 266754


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000


In [17]:
classification_res = pd.DataFrame(columns = cell_lines)
classification_res.loc['GCN'] = gcn_auroc
classification_res

,E116,E122,E123
GCN,0.912025,0.906868,0.92633


### MLP

In [18]:
max_epoch = 1000
mlp_auroc = []

for cell_line in cell_lines:

    train_idx = torch.load(f'./train-test-split/{cell_line}/train_idx.pt')
    valid_idx = torch.load(f'./train-test-split/{cell_line}/valid_idx.pt')
    test_idx = torch.load(f'./train-test-split/{cell_line}/test_idx.pt')
    
    print(f'\nTraining Cell Line {cell_line}...')
    G, targetNode_mask = prepare_data(cell_line=cell_line, regression_flag = regression_flag, base_path = src_dir)
    
    model = MLP_classification(num_feat, num_lin_layers, lin_hidden_sizes, num_classes)
    optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, model.parameters()), lr = learning_rate)
    
    train_loss_vec, train_AUROC_vec, valid_loss_vec, valid_AUROC_vec = \
    train_model_classification(model, G, max_epoch, learning_rate, targetNode_mask, train_idx, valid_idx, optimizer)

    test_AUROC, test_AUPR, test_pred, test_labels, train_AUPR, train_pred, train_labels, \
        valid_AUPR, valid_pred, valid_labels = \
            eval_model_classification(model, G, targetNode_mask, train_idx, valid_idx, test_idx)
    
    mlp_auroc.append(test_AUROC)


Training Cell Line E116...


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000

Training Cell Line E122...


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000

Training Cell Line E123...


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000


In [19]:
classification_res.loc['MLP'] = mlp_auroc
classification_res

,E116,E122,E123
GCN,0.912025,0.906868,0.926330
MLP,0.908909,0.896096,0.918319


### Weighted Aggregation GCN

In [20]:
max_epoch = 1000
w_gcn_auroc = []

for cell_line in cell_lines:
    print(f'\nTraining Cell Line {cell_line}...')

    train_idx = torch.load(f'train-test-split/{cell_line}/train_idx.pt')
    valid_idx = torch.load(f'train-test-split/{cell_line}/valid_idx.pt')
    test_idx = torch.load(f'train-test-split/{cell_line}/test_idx.pt')
    
    G, targetNode_mask = prepare_data(cell_line = cell_line, regression_flag = regression_flag, base_path = src_dir)
    
    model = GCN_classification_weighted(num_feat, num_graph_conv_layers, graph_conv_layer_sizes, num_lin_layers, graph_lin_hidden_sizes, num_classes, num_nodes=G.x.shape[0], edge_attr=G.edge_attr)
    optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, model.parameters()), lr = learning_rate)
    loss = nn.CrossEntropyLoss()

    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    print(f'Number of Parameters: {sum([np.prod(p.size()) for p in model_parameters])}')

    train_out = train_model_classification(model, G, max_epoch, learning_rate, targetNode_mask, train_idx, valid_idx, optimizer)
    test_AUROC, test_AUPR, test_pred, test_labels, train_AUPR, train_pred, train_labels, \
        valid_AUPR, valid_pred, valid_labels = eval_model_classification(model, G, targetNode_mask, train_idx, valid_idx, test_idx)

    w_gcn_auroc.append(test_AUROC)


Training Cell Line E116...
Number of Parameters: 8080582


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000

Training Cell Line E122...
Number of Parameters: 7091466


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000

Training Cell Line E123...
Number of Parameters: 7284882


Epoch 0 out of 1000
Epoch 100 out of 1000
Epoch 200 out of 1000
Epoch 300 out of 1000
Epoch 400 out of 1000
Epoch 500 out of 1000
Epoch 600 out of 1000
Epoch 700 out of 1000
Epoch 800 out of 1000
Epoch 900 out of 1000


In [22]:
classification_res.loc['Weighted_Agg_GCN'] = w_gcn_auroc
classification_res

,E116,E122,E123
GCN,0.912025,0.906868,0.926330
MLP,0.908909,0.896096,0.918319
Weighted Agg GCN,0.911979,0.907122,0.926355
Weighted_Agg_GCN,0.911979,0.907122,0.926355


### GAT

In [26]:
# resetting states

cell_line = 'E116'
max_epoch = 1000
learning_rate = 1e-4
num_graph_conv_layers = 2
graph_conv_embed_size = 256
num_lin_layers = 3
lin_hidden_size = 256
regression_flag = 0
random_seed = 0

chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)
num_classes = 2 if regression_flag == 0 else 1

src_dir = os.getcwd()
save_dir = os.path.join(src_dir, 'data', cell_line, 'saved_runs')
hic_sparse_mat_file = os.path.join(src_dir, 'data', cell_line, 'hic_sparse.npz')
np_nodes_lab_genes_file = os.path.join(src_dir, 'data',  cell_line, \
    'np_nodes_lab_genes_reg' + str(regression_flag) + '.npy')
np_hmods_norm_all_file = os.path.join(src_dir, 'data', cell_line, \
    'np_hmods_norm_chip_' + str(chip_res) + 'bp.npy')
df_genes_file = os.path.join(src_dir, 'data', cell_line, 'df_genes_reg' + str(regression_flag) + '.pkl')
df_genes = pd.read_pickle(df_genes_file)

mat = load_npz(hic_sparse_mat_file)
allNodes_hms = np.load(np_hmods_norm_all_file)
hms = allNodes_hms[:, 1:] #only includes features, not node ids
X = torch.tensor(hms).float().reshape(-1, num_feat) 
allNodes = allNodes_hms[:, 0].astype(int)
geneNodes_labs = np.load(np_nodes_lab_genes_file)

geneNodes = geneNodes_labs[:, -2].astype(int)
allLabs = -1*np.ones(np.shape(allNodes))

targetNode_mask = torch.tensor(geneNodes).long()

if regression_flag == 0:
    geneLabs = geneNodes_labs[:, -1].astype(int)
    allLabs[geneNodes] = geneLabs
    Y = torch.tensor(allLabs).long()
else:
    geneLabs = geneNodes_labs[:, -1].astype(float)
    allLabs[geneNodes] = geneLabs
    Y = torch.tensor(allLabs).float()

#hyperparameter and model definitions
max_epoch = 1000
learning_rate = 1e-4
num_graph_conv_layers = 2
graph_conv_embed_size = 256
num_lin_layers = 3
lin_hidden_size = 256
regression_flag = 0
random_seed = 0

chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)
num_classes = 2 if regression_flag == 0 else 1

print('ChIP-seq resolution:', str(chip_res))
print('\n')
print('Training set: 70%')
print('Validation set: 15%')
print('Testing set: 15%')
print('\n')
print('Model hyperparameters: ')
print('Number of epochs:', max_epoch)
print('Learning rate:', learning_rate)
print('Number of graph convolutional layers:', str(num_graph_conv_layers))
print('Graph convolutional embedding size:', graph_conv_embed_size)
print('Number of linear layers:', str(num_lin_layers))
print('Linear hidden layer size:', lin_hidden_size)

# random_seed = random.randint(0,10000)
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

###Test for GPU availability
cuda_flag = torch.cuda.is_available()
if cuda_flag:  
  dev = "cuda" 
else:
  dev = "cpu"  
device = torch.device(dev)

graph_conv_embed_size = 256
num_graph_conv_layers = 2

graph_conv_layer_sizes = [num_feat] + \
    [int(max(graph_conv_embed_size, lin_hidden_size)) \
          for i in np.arange(1, num_graph_conv_layers, 1)] + [lin_hidden_size]

graph_lin_hidden_sizes = [graph_conv_layer_sizes[-1]] + \
    [int(max(lin_hidden_size, num_classes)) \
          for i in np.arange(1, num_lin_layers, 1)] + [num_classes]

graph_lin_hidden_sizes_reg = [graph_conv_layer_sizes[-1]] + \
    [int(max(lin_hidden_size, num_classes)) \
          for i in np.arange(1, num_lin_layers, 1)] + [1]

num_classes_reg = 1
lin_hidden_sizes = [num_feat] + [int(max(lin_hidden_size, num_classes)) for i in np.arange(1, num_lin_layers, 1)] + [num_classes]
lin_hidden_sizes_reg = [num_feat] + [int(max(lin_hidden_size, num_classes_reg)) for i in np.arange(1, num_lin_layers, 1)] + [num_classes_reg]
cell_lines = ['E116', 'E122', 'E123']

print(lin_hidden_sizes)

ChIP-seq resolution: 10000


Training set: 70%
Validation set: 15%
Testing set: 15%


Model hyperparameters: 
Number of epochs: 1000
Learning rate: 0.0001
Number of graph convolutional layers: 2
Graph convolutional embedding size: 256
Number of linear layers: 3
Linear hidden layer size: 256
[6, 256, 256, 2]


In [27]:
extract = torch_geometric.utils.from_scipy_sparse_matrix(mat)

data = torch_geometric.data.Data(edge_index = extract[0], edge_attr = extract[1], x = X, y = Y)
G = data

num_heads = 4
learning_rate = 0.002
max_epoch = 1000
dropout = 0.1
loss = nn.CrossEntropyLoss()
hidden_channels=[6, 30]
wd = 1e-05

gat_auroc = []
for cell_line in cell_lines:
    print(f'\nTraining Cell Line {cell_line}...')

    train_idx = torch.load(f'train-test-split/{cell_line}/train_idx.pt')
    valid_idx = torch.load(f'train-test-split/{cell_line}/valid_idx.pt')
    test_idx = torch.load(f'train-test-split/{cell_line}/test_idx.pt')

    gat = GAT(in_channels = 6, hidden_channels = hidden_channels, num_heads = num_heads, dropout = dropout)
    
    optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, gat.parameters()), lr = learning_rate, weight_decay = wd)
    
    train_losses, valid_losses = train_model_classification_GAT(gat, loss, G, max_epoch, learning_rate, targetNode_mask, train_idx, valid_idx, optimizer)
    
    out = eval_model_classification_GAT(gat, G, targetNode_mask, train_idx, valid_idx, test_idx)
    
    gat_auroc.append(out['test_AUROC'])


Training Cell Line E116...


Epoch 0: Train Loss = 0.6946897506713867, Valid Loss = 0.6951975226402283
Epoch 100: Train Loss = 0.5264937877655029, Valid Loss = 0.5406813025474548
Epoch 200: Train Loss = 0.5013999938964844, Valid Loss = 0.523849368095398
Epoch 300: Train Loss = 0.47166815400123596, Valid Loss = 0.49847838282585144
Epoch 400: Train Loss = 0.4441436231136322, Valid Loss = 0.47495073080062866
Epoch 500: Train Loss = 0.43035799264907837, Valid Loss = 0.4615248143672943
Epoch 600: Train Loss = 0.4219149649143219, Valid Loss = 0.4527394473552704
Epoch 700: Train Loss = 0.41287437081336975, Valid Loss = 0.4497264623641968
Epoch 800: Train Loss = 0.40680357813835144, Valid Loss = 0.44435736536979675
Epoch 900: Train Loss = 0.4009764790534973, Valid Loss = 0.44255053997039795

Training Cell Line E122...


Epoch 0: Train Loss = 0.7350696921348572, Valid Loss = 0.735917866230011
Epoch 100: Train Loss = 0.5280822515487671, Valid Loss = 0.5432596802711487
Epoch 200: Train Loss = 0.

In [28]:
classification_res.loc['GAT'] = gat_auroc
classification_res

,E116,E122,E123
GCN,0.912025,0.906868,0.926330
MLP,0.908909,0.896096,0.918319
Weighted_Agg_GCN,0.911979,0.907122,0.926355
GAT,0.883976,0.885955,0.879030


In [29]:
# classification_res.to_csv('results.csv')

### GAT w/ Neighbors

In [29]:
# resetting states

cell_line = 'E116'
max_epoch = 1000
learning_rate = 1e-4
num_graph_conv_layers = 2
graph_conv_embed_size = 256
num_lin_layers = 3
lin_hidden_size = 256
regression_flag = 0
random_seed = 0

chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)
num_classes = 2 if regression_flag == 0 else 1

src_dir = os.getcwd()
save_dir = os.path.join(src_dir, 'data', cell_line, 'saved_runs')
hic_sparse_mat_file = os.path.join(src_dir, 'data', cell_line, 'hic_sparse.npz')
np_nodes_lab_genes_file = os.path.join(src_dir, 'data',  cell_line, \
    'np_nodes_lab_genes_reg' + str(regression_flag) + '.npy')
np_hmods_norm_all_file = os.path.join(src_dir, 'data', cell_line, \
    'np_hmods_norm_chip_' + str(chip_res) + 'bp.npy')
df_genes_file = os.path.join(src_dir, 'data', cell_line, 'df_genes_reg' + str(regression_flag) + '.pkl')
df_genes = pd.read_pickle(df_genes_file)

mat = load_npz(hic_sparse_mat_file)
allNodes_hms = np.load(np_hmods_norm_all_file)
hms = allNodes_hms[:, 1:] #only includes features, not node ids
X = torch.tensor(hms).float().reshape(-1, num_feat) 
allNodes = allNodes_hms[:, 0].astype(int)
geneNodes_labs = np.load(np_nodes_lab_genes_file)

geneNodes = geneNodes_labs[:, -2].astype(int)
allLabs = -1*np.ones(np.shape(allNodes))

targetNode_mask = torch.tensor(geneNodes).long()

if regression_flag == 0:
    geneLabs = geneNodes_labs[:, -1].astype(int)
    allLabs[geneNodes] = geneLabs
    Y = torch.tensor(allLabs).long()
else:
    geneLabs = geneNodes_labs[:, -1].astype(float)
    allLabs[geneNodes] = geneLabs
    Y = torch.tensor(allLabs).float()

#hyperparameter and model definitions
max_epoch = 1000
learning_rate = 1e-4
num_graph_conv_layers = 2
graph_conv_embed_size = 256
num_lin_layers = 3
lin_hidden_size = 256
regression_flag = 0
random_seed = 0

chip_res = 10000
hic_res = 10000
num_hm = 6
num_feat = int((hic_res/chip_res)*num_hm)
num_classes = 2 if regression_flag == 0 else 1

print('ChIP-seq resolution:', str(chip_res))
print('\n')
print('Training set: 70%')
print('Validation set: 15%')
print('Testing set: 15%')
print('\n')
print('Model hyperparameters: ')
print('Number of epochs:', max_epoch)
print('Learning rate:', learning_rate)
print('Number of graph convolutional layers:', str(num_graph_conv_layers))
print('Graph convolutional embedding size:', graph_conv_embed_size)
print('Number of linear layers:', str(num_lin_layers))
print('Linear hidden layer size:', lin_hidden_size)

# random_seed = random.randint(0,10000)
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

###Test for GPU availability
cuda_flag = torch.cuda.is_available()
if cuda_flag:  
  dev = "cuda" 
else:
  dev = "cpu"  
device = torch.device(dev)

graph_conv_embed_size = 256
num_graph_conv_layers = 2

graph_conv_layer_sizes = [num_feat] + \
    [int(max(graph_conv_embed_size, lin_hidden_size)) \
          for i in np.arange(1, num_graph_conv_layers, 1)] + [lin_hidden_size]

graph_lin_hidden_sizes = [graph_conv_layer_sizes[-1]] + \
    [int(max(lin_hidden_size, num_classes)) \
          for i in np.arange(1, num_lin_layers, 1)] + [num_classes]

graph_lin_hidden_sizes_reg = [graph_conv_layer_sizes[-1]] + \
    [int(max(lin_hidden_size, num_classes)) \
          for i in np.arange(1, num_lin_layers, 1)] + [1]

num_classes_reg = 1
lin_hidden_sizes = [num_feat] + [int(max(lin_hidden_size, num_classes)) for i in np.arange(1, num_lin_layers, 1)] + [num_classes]
lin_hidden_sizes_reg = [num_feat] + [int(max(lin_hidden_size, num_classes_reg)) for i in np.arange(1, num_lin_layers, 1)] + [num_classes_reg]
cell_lines = ['E116', 'E122', 'E123']

print(lin_hidden_sizes)

ChIP-seq resolution: 10000


Training set: 70%
Validation set: 15%
Testing set: 15%


Model hyperparameters: 
Number of epochs: 1000
Learning rate: 0.0001
Number of graph convolutional layers: 2
Graph convolutional embedding size: 256
Number of linear layers: 3
Linear hidden layer size: 256
[6, 256, 256, 2]


In [33]:
num_heads = 4
learning_rate = 0.001
max_epoch = 20
loss = nn.CrossEntropyLoss()
hidden_channels=[6, 30]
wd = 1e-05

gat_auroc_neighbors = []
for cell_line in cell_lines:
    print(f'\nTraining Cell Line {cell_line}...')

    train_idx = torch.load(f'train-test-split/{cell_line}/train_idx.pt')
    valid_idx = torch.load(f'train-test-split/{cell_line}/valid_idx.pt')
    test_idx = torch.load(f'train-test-split/{cell_line}/test_idx.pt')

    train_n_loader = NeighborLoader(G, num_neighbors = [10, 10], batch_size = 64, input_nodes = targetNode_mask[train_idx], shuffle = True)
    valid_n_loader = NeighborLoader(G, num_neighbors = [10, 10], batch_size = 64, input_nodes = targetNode_mask[valid_idx], shuffle = False)

    gat = GAT(in_channels=6, hidden_channels=hidden_channels, num_heads = num_heads, dropout = 0.5)
    
    optimizer = torch.optim.Adam(filter(lambda p : p.requires_grad, gat.parameters()), lr = learning_rate, weight_decay = wd)
    
    train_losses, valid_losses = train_model_classification_GAT_Neighbors(gat, loss, train_n_loader, valid_n_loader, max_epoch, optimizer, train_idx, valid_idx)
    
    out = eval_model_classification_GAT(gat, G, targetNode_mask, train_idx, valid_idx, test_idx)
    
    gat_aurocs_neighbors.append(out['test_AUROC'])


Training Cell Line E116...
Epoch 0: Train Loss = 0.49647271633148193, Valid Loss = 0.5472369194030762
Epoch 1: Train Loss = 0.4698091447353363, Valid Loss = 0.5677899718284607
Epoch 2: Train Loss = 0.44664162397384644, Valid Loss = 0.637824535369873
Epoch 3: Train Loss = 0.41795074939727783, Valid Loss = 0.6678191423416138
Epoch 4: Train Loss = 0.5045174956321716, Valid Loss = 0.5813190937042236


KeyboardInterrupt: 